In [1]:
# Dynamic Parking Pricing - Google Colab Notebook

# Import libraries
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

# Load dataset
file_path = 'dataset(1).csv'  # Adjust if needed

df = pd.read_csv(file_path)

# Base parameters
BASE_PRICE = 10
ALPHA = 2

# ------------------------
# Model 1: Baseline Pricing
# ------------------------
def compute_baseline_price(df, base_price=BASE_PRICE, alpha=ALPHA):
    df = df.copy()
    df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']
    df['BaselinePrice'] = base_price + alpha * df['OccupancyRatio']
    return df

df = compute_baseline_price(df)

# --------------------------
# Model 2: Demand-Based Pricing
# --------------------------
vehicle_type_weight = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
traffic_level_weight = {'low': 0.5, 'medium': 1.0, 'high': 1.5}

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def compute_demand_based_price(df):
    df = df.copy()
    df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']
    df['VehicleWeight'] = df['VehicleType'].map(vehicle_type_weight)
    df['TrafficWeight'] = df['TrafficConditionNearby'].map(traffic_level_weight)

    df['DemandRaw'] = (
        1.0 * df['OccupancyRatio'] +
        0.3 * df['QueueLength'] -
        0.4 * df['TrafficWeight'] +
        0.8 * df['IsSpecialDay'] +
        0.5 * df['VehicleWeight']
    )

    df['NormalizedDemand'] = normalize(df['DemandRaw'])
    df['DemandPrice'] = BASE_PRICE * (1 + 0.5 * df['NormalizedDemand'])
    df['DemandPrice'] = df['DemandPrice'].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2)
    return df

df = compute_demand_based_price(df)

# -----------------------------
# Model 3: Competitive Pricing
# -----------------------------
def compute_competitor_price(df):
    df = df.copy()

    lot_stats = df.groupby('SystemCodeNumber').agg({
        'Latitude': 'mean',
        'Longitude': 'mean',
        'DemandPrice': 'mean'
    }).reset_index()

    competitor_prices = []
    reroute_flags = []

    for index, row in df.iterrows():
        lot_name = row['SystemCodeNumber']
        lot_latlon = (row['Latitude'], row['Longitude'])

        competitors = lot_stats[lot_stats['SystemCodeNumber'] != lot_name].copy()
        competitors['Distance'] = competitors.apply(
            lambda x: geodesic(lot_latlon, (x['Latitude'], x['Longitude'])).meters, axis=1
        )
        nearby = competitors[competitors['Distance'] <= 500]

        if not nearby.empty:
            avg_price = nearby['DemandPrice'].mean()
            reroute_flag = int(row['OccupancyRatio'] > 0.9 and avg_price < row['DemandPrice'])
        else:
            avg_price = np.nan
            reroute_flag = 0

        competitor_prices.append(avg_price)
        reroute_flags.append(reroute_flag)

    df['AvgCompetitorPrice'] = competitor_prices
    df['RerouteSuggested'] = reroute_flags

    df['CompetitivePrice'] = df['DemandPrice']
    mask = (df['AvgCompetitorPrice'].notnull()) & (df['AvgCompetitorPrice'] > df['DemandPrice'])
    df.loc[mask, 'CompetitivePrice'] = df.loc[mask, 'DemandPrice'] + 0.1 * (df.loc[mask, 'AvgCompetitorPrice'] - df.loc[mask, 'DemandPrice'])
    df['CompetitivePrice'] = df['CompetitivePrice'].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2)

    return df

df = compute_competitor_price(df)

# ------------------------
# Bokeh Visualization
# ------------------------
def plot_prices(df, lot='BHMBCCMKT01'):
    subset = df[df['SystemCodeNumber'] == lot].copy()
    subset['Time'] = pd.to_datetime(subset['LastUpdatedDate'] + ' ' + subset['LastUpdatedTime'], dayfirst=True)
    subset = subset.sort_values('Time')

    source = ColumnDataSource(subset)
    p = figure(x_axis_type="datetime", title=f"Dynamic Prices for {lot}", plot_height=350, plot_width=800)
    p.line(x='Time', y='BaselinePrice', source=source, line_width=2, color='green', legend_label="Baseline")
    p.line(x='Time', y='DemandPrice', source=source, line_width=2, color='blue', legend_label="Demand-Based")
    p.line(x='Time', y='CompetitivePrice', source=source, line_width=2, color='red', legend_label="Competitive")
    p.legend.location = "top_left"
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price"
    show(p)

plot_prices(df)

# Display sample data
print(df[['SystemCodeNumber', 'Occupancy', 'QueueLength', 'VehicleType', 'BaselinePrice', 'DemandPrice', 'CompetitivePrice', 'RerouteSuggested']].head())


ModuleNotFoundError: No module named 'geopy'